In [2]:
import sqlite3
import pandas as pd
import dspy
import dotenv, os
from pydantic import BaseModel, Field
import openai
import pm4py
import ast
import networkx as nx
from numpy import random
from dspy.evaluate import Evaluate
from collections import defaultdict
import tqdm as notebook_tqdm
import copy
import re
import traceback
from queue import Queue
import threading
from dspy.teleprompt import BootstrapFewShot
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
import phoenix as px
from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from chroma_retriever import Chroma
from chromadb.utils import embedding_functions
from contextlib import contextmanager
from dspy.primitives.assertions import assert_transform_module, backtrack_handler
import functools
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")

In [3]:
INPUT_FILE_NAME = "/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/dataset/Road_Traffic_Fine_Management_Process.xes" #replce with your file path
COLUMN_INSTRUCTIONS = "/Users/sulzair/Documents/Bachelor Thesis/Proof-of-Concept/Experiments/PM_EVALQUESTIONS_final.csv"
SQL_QUESTIONS = '/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/benchmark/sql_questions_to_splitt.csv'
SQLITE_DB_NAME = "combined.db" #"my_database.db" #leve as is
LLM_MODEL_TYPE = "gpt-4o" #gpt-4-turbo" #"gpt-3.5-turbo-0125" #leave as is for gpt 3.5 or change to "gpt-4-1106-preview" for gpt 4
PM_PY_PATH = "/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/Optimized_prompts/python/py_add_fewshot_12.json"
PM_SQL_PATH = "/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/Optimized_prompts/sql/sql_bootstrap_bootstrap_fewshot_1.json" # potentially we can go higher
JUDGE_PATH = "/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/Optimized_prompts/judge/judge_optimized_final.json"

In [169]:
from Utils.column_dependency import DependencyGraph

In [ ]:
phoenix_session = px.launch_app()
endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

In [ ]:
dotenv.load_dotenv()
lm = dspy.LM('openai/gpt-4o-mini', temperature=0.3, max_tokens=4096, stop=None, cache=False)
dspy.settings.configure(lm = lm)

In [ ]:
from collections import defaultdict
from pydantic import BaseModel
import dspy
from enum import Enum
from PY_programs.python_tables import PM_PY_no_deep
from SQL_programs.sql_reasoning import PM_SQL_multi_sp

class Decision(str, Enum):
    YES = "Answer with SQLite Query"
    NO = "Generate Additional Columns"

class binary_decision(BaseModel):
    decision: Decision

class Check(dspy.Signature):
    """Your job is to determine whether an additional column should be generated or if a question should be directly answered using a SQLite query, follow these heuristics:

    1. Repeated Complex Calculations: If a potential query involves repeated complex calculations or aggregations, we generate additional columns to store these pre-computed values. 
    This reduces the need for complex logic in the query itself.

    2. Data Completeness: Potential queries must account for all relevant cases, including those where certain events do not occur. 
    If a potential query might miss cases due to filtering, we pre-compute and store the necessary information in a new column.

    3. Simplify Aggregations: If a potential query requires multiple levels of aggregation (multiple boolean or other conditions), we generate columns that store intermediate results (boolean columns, or event counts) to simplify the final query.

    4. Minimize Logical Steps: We reduce the number of logical steps in potential queries by pre-computing values that require complex logic, 
    making the queries simpler and less error-prone.

    5. Readability and Maintainability: If a query becomes difficult to read and maintain due to its complexity, we generate additional columns. 
    Simpler queries are easier to debug and less likely to contain logical errors.

    By following these guidelines, we can ensure that our queries remain simple, maintainable, and accurate, reducing the likelihood of logical mistakes.
    If it appears that a question is referring to columns that are not present in the database, additional columns should allways be generated to provide the necessary information.
    This applies even to close matches (amount is present), but the questions is refering to amount_min or amount_last, which are not present in the database."""

    question = dspy.InputField()
    available_columns = dspy.InputField(desc="Information about the database and its tables")
    provided_reasoning = dspy.InputField(desc="Thinking that your colleague has done to arrive at the decision. You may use this to help you make your decision.")
    decision: binary_decision = dspy.OutputField(desc="Indicate whether the question can be answered directly using a SQLite query 'yes' or if additional columns should be generated 'no'.")

class Think(dspy.Signature):
    """Your job is to reason about whether an additional column should be generated or if a question should be directly answered using a SQLite query, follow these heuristics:

    1. Repeated Complex Calculations: If a potential query involves repeated complex calculations or aggregations, we generate additional columns to store these pre-computed values. 
    This reduces the need for complex logic in the query itself.

    2. Data Completeness: Potential queries must account for all relevant cases, including those where certain events do not occur. 
    If a potential query might miss cases due to filtering, we pre-compute and store the necessary information in a new column.

    3. Simplify Aggregations: If a potential query requires multiple levels of aggregation (multiple boolean or other conditions), we generate columns that store intermediate results (boolean columns, or event counts) to simplify the final query.

    4. Minimize Logical Steps: We reduce the number of logical steps in potential queries by pre-computing values that require complex logic, 
    making the queries simpler and less error-prone.

    5. Readability and Maintainability: If a query becomes difficult to read and maintain due to its complexity, we generate additional columns. 
    Simpler queries are easier to debug and less likely to contain logical errors.

    By following these guidelines, we can ensure that our queries remain simple, maintainable, and accurate, reducing the likelihood of logical mistakes.
    If it appears that a question is referring to columns that are not present in the database, additional columns should allways be generated to provide the necessary information.
    This applies even to close matches (amount is present), but the questions is refering to amount_min or amount_last, which are not present in the database.
    """
    question = dspy.InputField()
    available_columns = dspy.InputField(desc="Information about the database and its tables")
    reasoning = dspy.OutputField(desc="Reasoning about which decision to make based on the question and available columns.")


class PM_combined(dspy.Module):
    def __init__(self, dp_graph, rm, pool, conn_path, pm_py_path, pm_sql_path): # requires all the create column instructions to retrieve the correct instructions for missing cols
        super().__init__()
        self.dp_graph = dp_graph
        self.think = dspy.Predict(Think)
        self.check = dspy.Predict(Check)
        self.rm = rm
        self.pool = pool
        self.conn_path = conn_path
        self.pm_py = PM_PY_no_deep(rm = self.rm, conn_path= self.conn_path )
        self.pm_py.load(path=pm_py_path)
        self.pm_sql = PM_SQL_multi_sp(pool=self.pool, rm = self.rm)
        self.pm_sql.load(path=pm_sql_path)
        self.col_tacked = defaultdict(list)

    def forward(self, question, req_cols):

        self.question = question
        available_cols = self.rm.cols
        columns_to_generate = None
        if type(req_cols) == str:
            columns_to_generate = self.dp_graph.cols(req_cols[2:-2].split("', '"), available_cols)
        col_descriptions = self.rm.retrieve(question)
        thoughts = self.think(question = question, available_columns = col_descriptions)
        check = self.check(question = question, available_columns = col_descriptions, provided_reasoning = thoughts.reasoning)
        print("check", check.decision.decision)
        dspy.Suggest(
            check.decision.decision.lower() == "yes" or check.decision.decision.lower() == "no",
            "The response can only be 'yes' or 'no'.",
        )
        
        
        if check.decision.decision.lower() == "yes":
            print("calling sql module, question: ", question)
            try:
                if columns_to_generate and len(columns_to_generate) > 0:
                    self.col_tacked[question].append("FP")
                    print("Should have said NO, required columns to generate")
            except:
                self.col_tacked[question].append("TP")
                pass

            try:
                if not columns_to_generate or len(columns_to_generate) == 0:
                    self.col_tacked[question].append("TP")

            except:
                self.col_tacked[question].append("TP")
                pass
                    
            result = self.pm_sql(self.question)
            return result
            
        else:
            if columns_to_generate:
                print("there are column to generate")
                self.col_tacked[question].append("TN")
                instructions = self.dp_graph.instructions_c(columns_to_generate)
            #definitions = self.dp_graph.definitions_s(columns_to_generate)

                for instruction in instructions:
                    print("calling python module cols to generate: ", columns_to_generate)
                    
                    descript = self.pm_py(instruction) # if it fails, it should not write into rm that the column exists
                print("finished calling python, now calling sql module")
                result = self.pm_sql(question)
                return result
            else:
                print("it wanted to generate collumns for something that did not require it")
                print("calling sql module instead")
                self.col_tacked[question].append("FN")
                return self.pm_sql(question)
    def get_col_tracked(self):
        return self.col_tacked
        

Lets think this through 

So we know that with the graph we can kind of get the correct values based on whether or not additional columns must be generated.

This is basically our metric, is there a requirement for additional columns or not. Pretty straight forward.

The input is basically the columns that are available right now.

So given some random set of available columns we already know the correct answer.

All we need to do is come up with random columns to provide to the program.


We have to work with the training set, these will be basically only the columns in the training set for the python.


Here we can use us combined dataset, using the sql question (which are provided to it) along with the required column.

Additionally instead of using rm, we need to create these randomly. Either we use the vanilla ones, for quite a while, using the normal rm

or we kind of create a fake data set of cols, and add in a couple of additional columns 

lets take a look at the training set of the combined program.

In [4]:
qa = pd.read_csv(SQL_QUESTIONS)

In [5]:
trainset = []
testset = []
for category, question, answer, cols, split in qa.values:
    print(type(cols) == float)
    py = dspy.Example(question = question, example = answer, req_cols = cols).with_inputs("question", "req_cols")
    if split == "train":
        trainset.append(py)
    else:
        testset.append(py)

True
True
True
True
True
True
True
True
False
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False

In [8]:
for i in trainset:
    print(i.req_cols)

nan
nan
nan
nan
nan
['send_fine_count']
nan
['add_penalty_count']
['payment_count']
['payment_count']
['payment_count']
['payment_count']
['payment_count']
['payment_count']
['send_fine_count']
['send_fine_count']
['insert_fine_notification_count']
['insert_fine_notification_count']
['insert_fine_notification_count']
['send_for_credit_collection_count']
['send_for_credit_collection_count']
['insert_date_appeal_to_prefecture_count']
['insert_date_appeal_to_prefecture_count']
['send_appeal_to_prefecture_count']
['send_appeal_to_prefecture_count']
['receive_result_appeal_from_prefecture_count']
['notify_result_appeal_to_offender_count']
['appeal_to_judge_count']
['appeal_to_judge_count']
['event_count']
['event_count']
['event_count']
['event_count']
['event_count']
['event_count']
nan
['dismissed_by_other']
['dismissed_by_other']
['appeal_to_judgeorprefecture']
['appeal_to_judgeorprefecture']
['obligation_topay_cancelled']
['obligation_topay_cancelled']
['add_penalty_count']
['add_penalt

okay so we have a few that require nothing. Here I guess the edge case is providing the vanilla cols

Next come all the counts, we want to make sure that there is a little bit of diversity. So I guess we should try to maximizse the number of unique cols to add.


1. take all the nan, -> vanilla cols
2. take all the unique cols and maybe find a good question that is on the verge of being answerable.

3. for those we can do something like 60% do not have the required cols, and 40% already have the required cols.

In [1]:
rando_task = "Determine whether an additional column should be generated or if a question should be directly answered using a SQLite query, with a cautious approach as per the following heuristics:\n\n- **Repeated Complex Calculations**: Generate additional columns if potential queries involve repeated calculations or aggregations to store pre-computed values. This reduces complex logic in the query.\n\n- **Data Completeness**: Ensure queries account for all relevant cases, including where certain events do not occur. Pre-compute and store necessary information in a new column if a query might miss cases or events due to filtering.\n\n- **Simplify Aggregations**: Generate columns to store intermediate results (like boolean columns or event counts) if a query requires aggregation, simplifying the final query.\n\n- **Minimize Logical Steps**: Pre-compute values requiring logic to reduce logical steps in queries, making them simpler and less error-prone.\n\n- **Readability and Maintainability**: Generate additional columns if a query becomes difficult to read and maintain due to their length. Simpler queries are easier to debug and less likely to contain logical errors.\n\n- **Risk of Direct Queries**: Only opt for a SQL query directly if it is safe and extremely unlikely to cause an error or logical mistake. Generating additional columns is always a safe bet with no negative consequences. We want to avoid failure as much as possible. Even if we have all the columns required to write a SQLite query, it could be safer to outsource any additional operations to an expert column generator.\n\n- **Missing Columns**: Generate additional columns if questions refer to columns not present in the database, even with close matches (e.g., \"amount_min\" or \"amount_last\" if only \"amount\" is present).\n\n# Steps\n\n1. Analyze the complexity of the potential query and identify any repeated calculations or intricate logic.\n2. Check if the potential query covers all relevant data cases, including unoccurring events.\n3. Consider the aggregation levels involved and simplify by pre-computing when necessary.\n4. Evaluate the logical steps required in the query and minimize them by generating additional columns.\n5. Assess the query for readability and maintenance concerns, opting for column generation if it enhances clarity.\n6. Verify the presence of necessary columns in the database; generate columns for missing ones referenced in the question.\n7. Critically evaluate the risk of directly answering with a SQL query and ensure it is only chosen when truly safe.\n\n# Output Format\n\nProvide a detailed explanation of your assessment followed by your decision: \"Generate Additional Columns\" or \"Answer with SQLite Query\".\n\n# Notes\n\n- If the required column is not present in the database, always opt for generating additional columns.\n- Consider the maintainability and readability of queries as a factor for decision-making.\n- Always provide a clear reasoning process before concluding to justify the decision effectively.\n- Be particularly cautious when deciding to opt for a direct SQL query and ensure all safety measures are thoroughly considered.\n- As a rule of thumb, anything related to money or amounts should be handled by generating additional columns to avoid any potential errors and discrepancies (really important not to take any risk)."

print(rando_task)

Determine whether an additional column should be generated or if a question should be directly answered using a SQLite query, with a cautious approach as per the following heuristics:

- **Repeated Complex Calculations**: Generate additional columns if potential queries involve repeated calculations or aggregations to store pre-computed values. This reduces complex logic in the query.

- **Data Completeness**: Ensure queries account for all relevant cases, including where certain events do not occur. Pre-compute and store necessary information in a new column if a query might miss cases or events due to filtering.

- **Simplify Aggregations**: Generate columns to store intermediate results (like boolean columns or event counts) if a query requires aggregation, simplifying the final query.

- **Minimize Logical Steps**: Pre-compute values requiring logic to reduce logical steps in queries, making them simpler and less error-prone.

- **Readability and Maintainability**: Generate additio

In [28]:
dataset = []

In [29]:
# getting the vanilla values

for i in trainset:
    if type(i.req_cols) is not str:
        dataset.append(i)

In [30]:
dataset

[Example({'question': 'How many cases are in the log?', 'example': '150370', 'req_cols': nan}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many Create Fine events occur?', 'example': '150370', 'req_cols': nan}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many Add penalty events occur?', 'example': '79860', 'req_cols': nan}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many Send Appeal to Prefecture event occur?', 'example': '4141', 'req_cols': nan}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many times do each of the event types occur in the log?', 'example': 'Create Fine 150370 Send Fine 103987 Insert Fine Notification 79860 Add penalty 79860 Payment 77601 Send for Credit Collection 59013 Insert Date Appeal to Prefecture 4188 Send Appeal to Prefecture 4141 Receive Result Appeal from Prefecture 999 Notify Result Appeal to Offender 896 Appeal to Judge 555', 'req_cols': nan}) (input_keys={'question', 

In [31]:
unique = []
for i in trainset:
    if i.req_cols not in unique:
        unique.append(i.req_cols)


In [32]:
unique

[nan,
 "['send_fine_count']",
 "['add_penalty_count']",
 "['payment_count']",
 "['insert_fine_notification_count']",
 "['send_for_credit_collection_count']",
 "['insert_date_appeal_to_prefecture_count']",
 "['send_appeal_to_prefecture_count']",
 "['receive_result_appeal_from_prefecture_count']",
 "['notify_result_appeal_to_offender_count']",
 "['appeal_to_judge_count']",
 "['event_count']",
 "['dismissed_by_other']",
 "['appeal_to_judgeorprefecture']",
 "['obligation_topay_cancelled']",
 "['duration']",
 "['amount_min']",
 "['time_timestamp_beginn', 'duration']",
 "['time_timestamp_beginn', 'time_timestamp_end']",
 "['time_timestamp_beginn']",
 "['time_timestamp', 'concept_name']",
 "['concept_name', 'time_timestamp', 'time_timestamp_beginn']",
 "['concept_name', 'time_timestamp', 'time_timestamp_end']"]

In [103]:
# getting the good questions from the uniqe values
send_fine_count = []
for i in trainset:
    if i.req_cols == unique[22]:
        send_fine_count.append(i)

send_fine_count

[Example({'question': 'How many Insert Date Appeal to Prefecture events occur within 15 days of the end of their associated case? ', 'example': '208', 'req_cols': "['concept_name', 'time_timestamp', 'time_timestamp_end']"}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many Add penalty events occur within 3 days of the end of their associated case?', 'example': '2774', 'req_cols': "['concept_name', 'time_timestamp', 'time_timestamp_end']"}) (input_keys={'question', 'req_cols'})]

In [70]:
send_fine_count[0]

Example({'question': 'How many cases have been dismissed by something other than the judge or the prefecture?', 'example': '504', 'req_cols': "['dismissed_by_other']"}) (input_keys={'question', 'req_cols'})

In [86]:
dataset_4 = []

In [100]:
dataset_4.append(send_fine_count[1])

In [111]:
len(dataset_4)

5

In [69]:
dataset_3 = []

In [83]:
dataset_3.append(send_fine_count[2])

In [109]:
len(dataset_3)

5

In [35]:
dataset_2 = []

In [66]:
dataset_2.append(send_fine_count[5])

In [107]:
len(dataset_2)

10

In [105]:
len(dataset)

9

Okay now we have dataset which is basically just the vanilla cols questions -> all are true : 9

Next we have dataset_2 which is all the counts: 10 -> we could do 50/50 splitt and provide the respective counts in the cols for those ones

Then we have dataset_3 which are the normal cols: 5 -> simply provide the vanilla cols for this one (all require additional cols)

Then we have dataset_4, here I want them to also require additional cols to be created, but with a twist by providing one of the required cols 5

So we have for SQL:

9 + 5 = 14

9x vanilla

And for cols:

5 + 5 + 5

We require for:

dataset: 9x vanilla
dataset_2: 5x vanilla, 5x with counts (each respectively)
dataset_3: 5x vanilla
dataset_4: 5x with special cols

we now create the special cols:



In [115]:
dataset_2

# only show half of the dataset
dataset_2[int(len(dataset_2)/2):]

[Example({'question': 'What is the percentage of cases with more than one Insert date appeal to Prefecture event?', 'example': '0', 'req_cols': "['insert_date_appeal_to_prefecture_count']"}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many cases have more than one Send appeal to prefecture event?', 'example': '0', 'req_cols': "['send_appeal_to_prefecture_count']"}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'What is the average number of Receive Result appeal from Prefecture events per case?', 'example': '0.00664', 'req_cols': "['receive_result_appeal_from_prefecture_count']"}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'What is the average number of Appeal to Judge events per case?', 'example': '0.00369', 'req_cols': "['appeal_to_judge_count']"}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many cases have 2 or less events per case?', 'example': '66756', 'req_cols': "['event_count']"}) (input_keys={'questio

In [136]:
dataset_4

[Example({'question': 'How many cases beginn before 2002-04-24 AND have a duration smaller than 5?', 'example': '455', 'req_cols': "['time_timestamp_beginn', 'duration']"}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many cases have a time_timestamp_beginn higher equal 2002-04-24 AND time_timestamp_end lower equal 2002-12-24?', 'example': '4607', 'req_cols': "['time_timestamp_beginn', 'time_timestamp_end']"}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many cases beginn after 2013-04-24?', 'example': '803', 'req_cols': "['time_timestamp_beginn']"}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many Send for Credit Collection events occur at most 290 days after the start of their associated case?', 'example': '12', 'req_cols': "['concept_name', 'time_timestamp', 'time_timestamp_beginn']"}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many Add penalty events occur within 3 days of the end of their associ

In [143]:
dataset_4[1].question

'How many cases have a time_timestamp_beginn higher equal 2002-04-24 AND time_timestamp_end lower equal 2002-12-24?'

In [113]:
# we take the last 5

dataset_2_special_cols = []



In [137]:
dataset_4_cols = []

In [184]:
# Initialize the Chroma retriever

col_desc = """- 'amount' (int): The amount due to be paid for the fine (including the penalty amount in case it is added). There are no nan values in this column.
- 'org_resource' (int): A numeric code indicating the employee who handled the case.
- 'dismissal' (string): A flag indicating whether and by whom the fine is dismissed. It is initialized to NIL. We know the meaning of:  
        'G': dismissed by the judge
        '#': dismissed by the prefecture
        NIL: not dismissed, i.e., to be paid.
        There are several other values used for which we do not know the semantics.
- 'vehicleClass' (string): A flag indicating the kind of vehicle driven or owned by the offender. The semantics of the values is unknown.
- 'totalPaymentAmount' (int): The cumulative amount paid by the offender. It is always initialized to 0. There are no nan values in this column.
- 'lifecycle_transition' (string): the transition of the activity (complete, start, etc.)
- 'article' (string): The number of the article of the Italian roadtraffic law that is violated by the offender (e.g., article 157 refers to stopping and parking vehicles).
- 'points' (float): Penalty points deducted from the driving license. In Italy, each driver starts with 20 points on their license and may loose points for each offence, based on the gravity.
- 'expense' (int): The additional amount due to be paid for postal expenses. There are no nan values in this column.
- 'notificationType' (string): A flag indicating to whom the fine refers. 'P': car owner, 'C': car driver.
- 'lastSent' (datetime): N/A
- 'paymentAmount' (int): The amount paid by the offender in one transaction. There are no nan values in this column.
- 'matricola' (string): N/A (Probably refers to the matriculation number of the car.)
- 'concept_name' (string): the activity/ event type name
        Activity Description, column: 'concept:name':
            > 'Create Fine': The initial creation of the fine in the information system. It initializes event log attributes amount, dismissal, points and totalPaymentAmount.
            > 'Send Fine': A notification about the fine is sent by post to the offender.
            > 'Insert Fine Notification': The notification is received by the offender.
            > 'Add penalty': An additional penalty is applied.
            > 'Payment': A payment made by the offender is registered.
            > 'Send for Credit Collection': Unpaid fines are sent for credit collection. A separate process is started by a collection agency to collect the money of the unpaid fines.
            > 'Insert Date Appeal to Prefecture': The offender appeals against the fine to the prefecture. A prefecture in Italy is an administrative body representing the national government in each province.
            > 'Send Appeal to Prefecture': The appeal is sent to the prefecture by the local police.
            > 'Receive Result Appeal from Prefecture': The local police receives the result of the appeal. If the prefecture dismisses the fine, the appeal is deemed accepted, and the obligation to pay the fine is cancelled. In this case, there is no need for the police to receive the result from the prefecture (Receive Result Appeal from Prefecture) and notify the offender (Notify Result Appeal to Offender).
            > 'Notify Result Appeal to Offender': The local police informs the offender of the appeal result. 
            > 'Appeal to Judge': The offender appeals against the fine to a judge.
        IMPORTANT: The last event in a case can be arbitrary. There is no guarantee that the last event is 'Send Fine' or 'Payment'. The last event can be any event in the log."""

rm = Chroma(sentence_transformer_ef = sentence_transformer_ef , documentation = col_desc) # for python, use standard cols

["- 'amount' (int): The amount due to be paid for the fine (including the penalty amount in case it is added). There are no nan values in this column.", "- 'org_resource' (int): A numeric code indicating the employee who handled the case.", "- 'dismissal' (string): A flag indicating whether and by whom the fine is dismissed. It is initialized to NIL. We know the meaning of:  \n        'G': dismissed by the judge\n        '#': dismissed by the prefecture\n        NIL: not dismissed, i.e., to be paid.\n        There are several other values used for which we do not know the semantics.", "- 'vehicleClass' (string): A flag indicating the kind of vehicle driven or owned by the offender. The semantics of the values is unknown.", "- 'totalPaymentAmount' (int): The cumulative amount paid by the offender. It is always initialized to 0. There are no nan values in this column.", "- 'lifecycle_transition' (string): the transition of the activity (complete, start, etc.)", "- 'article' (string): The

In [142]:
rm.add_new("- 'time_timestamp_beginn' (datetime): The timestamp of the first event for each case, consistent across all rows pertaining to the same case.")

In [151]:
dataset_4_cols.append(rm.retrieve(dataset_4[4].question))

In [152]:
dataset_4_cols

['THE DATABASE CONTAINS THE TABLE: event_log CONTAINING THE FOLLOWING COLUMNS (only the ones denoted by a "-", )\n- \'case_concept_name\' (string): the case identifier, use this to group by cases (retrieve information about cases as a whole)\n- \'time_timestamp\' (datetime): the timestamp of the activity.\n- \'duration\' (int): The total duration in seconds between the first and last event for each case, consistent across all rows pertaining to the same case.\n- \'event_count\' (int): The number of events recorded for each case, consistent across all rows pertaining to the same case.\n- \'appeal_to_judge_count\' (int): The number of times the event \'Appeal to Judge\' occurs for each case, consistent across all rows pertaining to the same case.\n- \'concept_name\' (string): the activity/ event type name\n        Activity Description, column: \'concept:name\':\n            > \'Create Fine\': The initial creation of the fine in the information system. It initializes event log attributes 

In [170]:
dataset_4[0].question

'How many cases beginn before 2002-04-24 AND have a duration smaller than 5?'

In [172]:
print(dataset_4_cols[0])

THE DATABASE CONTAINS THE TABLE: event_log CONTAINING THE FOLLOWING COLUMNS (only the ones denoted by a "-", )
- 'case_concept_name' (string): the case identifier, use this to group by cases (retrieve information about cases as a whole)
- 'time_timestamp' (datetime): the timestamp of the activity.
- 'duration' (int): The total duration in seconds between the first and last event for each case, consistent across all rows pertaining to the same case.
- 'event_count' (int): The number of events recorded for each case, consistent across all rows pertaining to the same case.
- 'appeal_to_judge_count' (int): The number of times the event 'Appeal to Judge' occurs for each case, consistent across all rows pertaining to the same case.
- 'concept_name' (string): the activity/ event type name
        Activity Description, column: 'concept:name':
            > 'Create Fine': The initial creation of the fine in the information system. It initializes event log attributes amount, dismissal, points an

In [134]:
dataset_2_special_cols.append(rm.retrieve("How many cases have 2 or less events per case?"))

In [135]:
dataset_2_special_cols

['THE DATABASE CONTAINS THE TABLE: event_log CONTAINING THE FOLLOWING COLUMNS (only the ones denoted by a "-", )\n- \'case_concept_name\' (string): the case identifier, use this to group by cases (retrieve information about cases as a whole)\n- \'time_timestamp\' (datetime): the timestamp of the activity.\n- \'insert_date_appeal_to_prefecture_count\' (int): The number of times the event \'Insert Date Appeal to Prefecture\' occurs for each case, consistent across all rows pertaining to the same case.\n- \'concept_name\' (string): the activity/ event type name\n        Activity Description, column: \'concept:name\':\n            > \'Create Fine\': The initial creation of the fine in the information system. It initializes event log attributes amount, dismissal, points and totalPaymentAmount.\n            > \'Send Fine\': A notification about the fine is sent by post to the offender.\n            > \'Insert Fine Notification\': The notification is received by the offender.\n            > \

In [ ]:
okay so now we have 

okay so now we have the second half of the cols for dataset_2_special_cols, and we have the complete dataset_4 cols. 

What we do now is iterate over the datasets and append the default cols to then, given the question and we create a singular dataset, so making the dataset with the questions etc and one with the columns. then what we will do is create a proper dspy dataset from that and start working on our little program.



In [154]:
final_dataset = []
final_cols = []

In [153]:
for i in dataset:
    print(i.question)

How many cases are in the log?
How many Create Fine events occur?
How many Add penalty events occur?
How many Send Appeal to Prefecture event occur?
How many times do each of the event types occur in the log?
How many cases are Sent for Credit Collection?
How many vehicle classes are there?
What are the top 10 cases with the highest single payment amount?
Which case has the highest signle totalPaymentAmount?


In [155]:
for i in dataset:
    final_dataset.append(i)
    final_cols.append(rm.retrieve(i.question))

In [159]:
for i in dataset_2[:int(len(dataset_2)/2)]:
    final_dataset.append(i)
    final_cols.append(rm.retrieve(i.question))

In [160]:
for i in dataset_2[int(len(dataset_2)/2):]:
    final_dataset.append(i)

In [161]:
for i in dataset_2_special_cols:
    final_cols.append(i)

In [164]:
for i in dataset_3:
    final_dataset.append(i)
    final_cols.append(rm.retrieve(i.question))

In [165]:
for i in dataset_4:
    final_dataset.append(i)

for p in dataset_4_cols:
    final_cols.append(p)


In [166]:
final_dataset

[Example({'question': 'How many cases are in the log?', 'example': '150370', 'req_cols': nan}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many Create Fine events occur?', 'example': '150370', 'req_cols': nan}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many Add penalty events occur?', 'example': '79860', 'req_cols': nan}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many Send Appeal to Prefecture event occur?', 'example': '4141', 'req_cols': nan}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many times do each of the event types occur in the log?', 'example': 'Create Fine 150370 Send Fine 103987 Insert Fine Notification 79860 Add penalty 79860 Payment 77601 Send for Credit Collection 59013 Insert Date Appeal to Prefecture 4188 Send Appeal to Prefecture 4141 Receive Result Appeal from Prefecture 999 Notify Result Appeal to Offender 896 Appeal to Judge 555', 'req_cols': nan}) (input_keys={'question', 

In [167]:
final_cols

['THE DATABASE CONTAINS THE TABLE: event_log CONTAINING THE FOLLOWING COLUMNS (only the ones denoted by a "-", )\n- \'case_concept_name\' (string): the case identifier, use this to group by cases (retrieve information about cases as a whole)\n- \'time_timestamp\' (datetime): the timestamp of the activity.\n- \'org_resource\' (int): A numeric code indicating the employee who handled the case.\n- \'concept_name\' (string): the activity/ event type name\n        Activity Description, column: \'concept:name\':\n            > \'Create Fine\': The initial creation of the fine in the information system. It initializes event log attributes amount, dismissal, points and totalPaymentAmount.\n            > \'Send Fine\': A notification about the fine is sent by post to the offender.\n            > \'Insert Fine Notification\': The notification is received by the offender.\n            > \'Add penalty\': An additional penalty is applied.\n            > \'Payment\': A payment made by the offender i

Now we adjust this thing:

    py = dspy.Example(question = question, example = answer, req_cols = cols).with_inputs("question", "req_cols")

    so that the question is used from the final_dataset, the answer will be taken from the graph thing, and then the req_cols is also just the same, we will also add in the column description as an input

In [168]:
new_dataset = []

for t in range(len(final_dataset)):

    example = dspy.Example(question = final_dataset[t].question, example = answer, column_description = final_cols[t]).with_inputs("question", "column_description")
    print(final_dataset[t].question)
    print(final_cols[t])

How many cases are in the log?
THE DATABASE CONTAINS THE TABLE: event_log CONTAINING THE FOLLOWING COLUMNS (only the ones denoted by a "-", )
- 'case_concept_name' (string): the case identifier, use this to group by cases (retrieve information about cases as a whole)
- 'time_timestamp' (datetime): the timestamp of the activity.
- 'org_resource' (int): A numeric code indicating the employee who handled the case.
- 'concept_name' (string): the activity/ event type name
        Activity Description, column: 'concept:name':
            > 'Create Fine': The initial creation of the fine in the information system. It initializes event log attributes amount, dismissal, points and totalPaymentAmount.
            > 'Send Fine': A notification about the fine is sent by post to the offender.
            > 'Insert Fine Notification': The notification is received by the offender.
            > 'Add penalty': An additional penalty is applied.
            > 'Payment': A payment made by the offender is

Hard coding this:




In [173]:
dataset_4[0].question

'How many cases beginn before 2002-04-24 AND have a duration smaller than 5?'

In [174]:
dataset_4_cols[0]

'THE DATABASE CONTAINS THE TABLE: event_log CONTAINING THE FOLLOWING COLUMNS (only the ones denoted by a "-", )\n- \'case_concept_name\' (string): the case identifier, use this to group by cases (retrieve information about cases as a whole)\n- \'time_timestamp\' (datetime): the timestamp of the activity.\n- \'duration\' (int): The total duration in seconds between the first and last event for each case, consistent across all rows pertaining to the same case.\n- \'event_count\' (int): The number of events recorded for each case, consistent across all rows pertaining to the same case.\n- \'appeal_to_judge_count\' (int): The number of times the event \'Appeal to Judge\' occurs for each case, consistent across all rows pertaining to the same case.\n- \'concept_name\' (string): the activity/ event type name\n        Activity Description, column: \'concept:name\':\n            > \'Create Fine\': The initial creation of the fine in the information system. It initializes event log attributes a

Answer:

In [175]:
first_answer= """To determine how many cases begin before 2002-04-24 and have a duration smaller than 5, we need to consider the following:

1. **Complexity of the Query**: The query requires identifying the start date of each case, which involves finding the earliest timestamp for each case. This is a repeated calculation since it must be done for every case.

2. **Data Completeness**: We need to ensure that we correctly identify the start of each case. This requires examining all events for each case to find the minimum timestamp, which is a complex operation if done repeatedly in a query.

3. **Aggregation and Simplification**: The task involves aggregating data at the case level (finding the earliest timestamp and checking the duration). Pre-computing the start date for each case simplifies the query by reducing it to a simple filter operation.

4. **Logical Steps**: By generating an additional column for the start date of each case, we reduce the logical complexity of the query. Instead of calculating the minimum timestamp within the query, we can directly filter on the pre-computed start date.

5. **Readability and Maintainability**: A query that directly calculates the minimum timestamp for each case would be more complex and harder to maintain. Pre-computing the start date makes the query more straightforward and easier to understand.

6. **Presence of Necessary Columns**: The existing columns do not include a pre-computed start date for each case. We need to generate this column to facilitate the query.

Given these considerations, it is clear that generating an additional column for the start date of each case is the most efficient and error-free approach. This column will store the earliest timestamp for each case, allowing us to easily filter cases that begin before a specific date.

**Decision**: Generate Additional Columns

This approach ensures that our query is simple, efficient, and maintains data integrity by accurately capturing the start date of each case."""

In [176]:
first_answer

'To determine how many cases begin before 2002-04-24 and have a duration smaller than 5, we need to consider the following:\n\n1. **Complexity of the Query**: The query requires identifying the start date of each case, which involves finding the earliest timestamp for each case. This is a repeated calculation since it must be done for every case.\n\n2. **Data Completeness**: We need to ensure that we correctly identify the start of each case. This requires examining all events for each case to find the minimum timestamp, which is a complex operation if done repeatedly in a query.\n\n3. **Aggregation and Simplification**: The task involves aggregating data at the case level (finding the earliest timestamp and checking the duration). Pre-computing the start date for each case simplifies the query by reducing it to a simple filter operation.\n\n4. **Logical Steps**: By generating an additional column for the start date of each case, we reduce the logical complexity of the query. Instead o

In [182]:
dataset_3

[Example({'question': 'How many cases have been dismissed by something other than the judge or the prefecture?', 'example': '504', 'req_cols': "['dismissed_by_other']"}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'What is the precentage of cases which have appealed to either the judge or the prefecture?', 'example': '0.03029 or 3.029%', 'req_cols': "['appeal_to_judgeorprefecture']"}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'How many cases as the obligation to pay been cancelled?', 'example': '49772', 'req_cols': "['obligation_topay_cancelled']"}) (input_keys={'question', 'req_cols'}),
 Example({'question': 'What are the top 10 cases with the longest duration(in days)?', 'example': 'V3904:4372 days\r\nN22795: 4352 days\r\nV4290: 4341 days\r\nS38092:4339 days\r\nV4358: 4320 days\r\nV4425:4308 days\r\nV5141: 4217 days\r\nV5285: 4212 days\r\nV5624: 4202 days\r\nV6005: 4176 days\r\nV6555: 4140 days', 'req_cols': "['duration']"}) (input_keys={'question'

In [183]:
dataset_3[-1].question

'What is the lowest amount min found in the event log?'

In [185]:
col_second_question = rm.retrieve(dataset_3[-1].question)

In [186]:
col_second_question

'THE DATABASE CONTAINS THE TABLE: event_log CONTAINING THE FOLLOWING COLUMNS (only the ones denoted by a "-", )\n- \'case_concept_name\' (string): the case identifier, use this to group by cases (retrieve information about cases as a whole)\n- \'time_timestamp\' (datetime): the timestamp of the activity.\n- \'amount\' (int): The amount due to be paid for the fine (including the penalty amount in case it is added). There are no nan values in this column.\n- \'lastSent\' (datetime): N/A\n- \'paymentAmount\' (int): The amount paid by the offender in one transaction. There are no nan values in this column.\n- \'concept_name\' (string): the activity/ event type name\n        Activity Description, column: \'concept:name\':\n            > \'Create Fine\': The initial creation of the fine in the information system. It initializes event log attributes amount, dismissal, points and totalPaymentAmount.\n            > \'Send Fine\': A notification about the fine is sent by post to the offender.\n 

In [187]:
print(col_second_question)

THE DATABASE CONTAINS THE TABLE: event_log CONTAINING THE FOLLOWING COLUMNS (only the ones denoted by a "-", )
- 'case_concept_name' (string): the case identifier, use this to group by cases (retrieve information about cases as a whole)
- 'time_timestamp' (datetime): the timestamp of the activity.
- 'amount' (int): The amount due to be paid for the fine (including the penalty amount in case it is added). There are no nan values in this column.
- 'lastSent' (datetime): N/A
- 'paymentAmount' (int): The amount paid by the offender in one transaction. There are no nan values in this column.
- 'concept_name' (string): the activity/ event type name
        Activity Description, column: 'concept:name':
            > 'Create Fine': The initial creation of the fine in the information system. It initializes event log attributes amount, dismissal, points and totalPaymentAmount.
            > 'Send Fine': A notification about the fine is sent by post to the offender.
            > 'Insert Fine Not

In [189]:
second_answer = """To determine the lowest amount min found in the event log, we need to consider the following:

1. **Complexity of the Query**: The task requires finding the minimum amount per case, which involves grouping by `case_concept_name` and then finding the minimum `amount` for each group. This is a relatively simple aggregation but can become complex if done repeatedly across multiple queries.

2. **Data Completeness**: The query should account for all cases, ensuring that no case is missed. By pre-computing the minimum amount per case, we ensure that every case is considered, even if it has only one event.

3. **Simplify Aggregations**: By generating an additional column to store the minimum amount per case, we simplify any future queries that might need this information. This avoids repeating the aggregation logic in multiple queries.

4. **Minimize Logical Steps**: Pre-computing the minimum amount per case reduces the logical steps needed in future queries, making them simpler and less error-prone.

5. **Readability and Maintainability**: Storing the minimum amount per case in a new column enhances the readability and maintainability of future queries, as they can directly reference this column instead of recalculating it.

6. **Risk of Direct Queries**: While it is possible to directly query the minimum amount using SQL, generating an additional column is a safer approach. It avoids potential errors in complex queries and ensures that the data is readily available for any future analysis.

Given these considerations, it is prudent to generate an additional column that stores the minimum amount per case. This approach aligns with best practices for simplifying queries, ensuring data completeness, and enhancing maintainability.

**Decision**: Generate Additional Columns"""

In [190]:
second_answer

'To determine the lowest amount min found in the event log, we need to consider the following:\n\n1. **Complexity of the Query**: The task requires finding the minimum amount per case, which involves grouping by `case_concept_name` and then finding the minimum `amount` for each group. This is a relatively simple aggregation but can become complex if done repeatedly across multiple queries.\n\n2. **Data Completeness**: The query should account for all cases, ensuring that no case is missed. By pre-computing the minimum amount per case, we ensure that every case is considered, even if it has only one event.\n\n3. **Simplify Aggregations**: By generating an additional column to store the minimum amount per case, we simplify any future queries that might need this information. This avoids repeating the aggregation logic in multiple queries.\n\n4. **Minimize Logical Steps**: Pre-computing the minimum amount per case reduces the logical steps needed in future queries, making them simpler and

In [191]:
dataset_3[2].question

'How many cases as the obligation to pay been cancelled?'

In [192]:
col_third_question = rm.retrieve(dataset_3[2].question)

In [193]:
col_third_question

'THE DATABASE CONTAINS THE TABLE: event_log CONTAINING THE FOLLOWING COLUMNS (only the ones denoted by a "-", )\n- \'case_concept_name\' (string): the case identifier, use this to group by cases (retrieve information about cases as a whole)\n- \'time_timestamp\' (datetime): the timestamp of the activity.\n- \'concept_name\' (string): the activity/ event type name\n        Activity Description, column: \'concept:name\':\n            > \'Create Fine\': The initial creation of the fine in the information system. It initializes event log attributes amount, dismissal, points and totalPaymentAmount.\n            > \'Send Fine\': A notification about the fine is sent by post to the offender.\n            > \'Insert Fine Notification\': The notification is received by the offender.\n            > \'Add penalty\': An additional penalty is applied.\n            > \'Payment\': A payment made by the offender is registered.\n            > \'Send for Credit Collection\': Unpaid fines are sent for cr

In [194]:
print(col_third_question)

THE DATABASE CONTAINS THE TABLE: event_log CONTAINING THE FOLLOWING COLUMNS (only the ones denoted by a "-", )
- 'case_concept_name' (string): the case identifier, use this to group by cases (retrieve information about cases as a whole)
- 'time_timestamp' (datetime): the timestamp of the activity.
- 'concept_name' (string): the activity/ event type name
        Activity Description, column: 'concept:name':
            > 'Create Fine': The initial creation of the fine in the information system. It initializes event log attributes amount, dismissal, points and totalPaymentAmount.
            > 'Send Fine': A notification about the fine is sent by post to the offender.
            > 'Insert Fine Notification': The notification is received by the offender.
            > 'Add penalty': An additional penalty is applied.
            > 'Payment': A payment made by the offender is registered.
            > 'Send for Credit Collection': Unpaid fines are sent for credit collection. A separate pro

In [195]:
third_asnwer = """To determine how many cases have had the obligation to pay cancelled, we need to identify cases where the fine was dismissed by either the prefecture or the judge. This involves checking the 'dismissal' column for specific values ('G' for judge and '#' for prefecture) that indicate the cancellation of the obligation to pay.

### Analysis:

1. **Complexity of Query**:
   - The query involves checking multiple events within each case to determine if any event led to the dismissal of the fine. This requires grouping by 'case_concept_name' and checking the 'dismissal' column for specific values.
   - The logic to determine if a case is dismissed is straightforward but needs to be applied across potentially large datasets, making it repetitive.

2. **Data Completeness**:
   - The query must ensure that all cases are considered, even those with no dismissal events. This requires a comprehensive check across all events within each case.

3. **Simplification of Aggregations**:
   - By creating a boolean column indicating whether a case has been dismissed, we simplify the aggregation process. This avoids the need to repeatedly check the 'dismissal' column for each event in a case.

4. **Minimizing Logical Steps**:
   - Pre-computing a dismissal status for each case reduces the logical steps in the query, making it simpler and less prone to errors.

5. **Readability and Maintainability**:
   - Generating an additional column for dismissal status enhances query readability and maintainability. It allows future queries to simply count cases with a dismissal status without re-evaluating each event.

6. **Risk of Direct Queries**:
   - Directly querying the database without pre-computed columns increases the risk of missing cases or making logical errors, especially if the dataset is large or complex.

7. **Presence of Necessary Columns**:
   - The necessary 'dismissal' column is present, but the logic to determine case-level dismissal is not directly available, necessitating an additional column.

### Conclusion:

Given the complexity and potential for errors in directly querying each event to determine case-level dismissal, it is safer and more efficient to generate an additional column that indicates whether the obligation to pay has been cancelled for each case. This approach simplifies the logic, ensures completeness, and enhances maintainability.

**Decision: Generate Additional Columns**"""

In [196]:
third_asnwer

"To determine how many cases have had the obligation to pay cancelled, we need to identify cases where the fine was dismissed by either the prefecture or the judge. This involves checking the 'dismissal' column for specific values ('G' for judge and '#' for prefecture) that indicate the cancellation of the obligation to pay.\n\n### Analysis:\n\n1. **Complexity of Query**:\n   - The query involves checking multiple events within each case to determine if any event led to the dismissal of the fine. This requires grouping by 'case_concept_name' and checking the 'dismissal' column for specific values.\n   - The logic to determine if a case is dismissed is straightforward but needs to be applied across potentially large datasets, making it repetitive.\n\n2. **Data Completeness**:\n   - The query must ensure that all cases are considered, even those with no dismissal events. This requires a comprehensive check across all events within each case.\n\n3. **Simplification of Aggregations**:\n   

In [199]:
dataset[6].question

'How many vehicle classes are there?'

In [200]:
col_fourth_question = rm.retrieve(dataset[6].question)

In [201]:
col_fourth_question

'THE DATABASE CONTAINS THE TABLE: event_log CONTAINING THE FOLLOWING COLUMNS (only the ones denoted by a "-", )\n- \'case_concept_name\' (string): the case identifier, use this to group by cases (retrieve information about cases as a whole)\n- \'time_timestamp\' (datetime): the timestamp of the activity.\n- \'vehicleClass\' (string): A flag indicating the kind of vehicle driven or owned by the offender. The semantics of the values is unknown.\n- \'matricola\' (string): N/A (Probably refers to the matriculation number of the car.)\n- \'notificationType\' (string): A flag indicating to whom the fine refers. \'P\': car owner, \'C\': car driver.\n- \'article\' (string): The number of the article of the Italian roadtraffic law that is violated by the offender (e.g., article 157 refers to stopping and parking vehicles).\n- \'points\' (float): Penalty points deducted from the driving license. In Italy, each driver starts with 20 points on their license and may loose points for each offence, b

In [202]:
print(col_fourth_question)

THE DATABASE CONTAINS THE TABLE: event_log CONTAINING THE FOLLOWING COLUMNS (only the ones denoted by a "-", )
- 'case_concept_name' (string): the case identifier, use this to group by cases (retrieve information about cases as a whole)
- 'time_timestamp' (datetime): the timestamp of the activity.
- 'vehicleClass' (string): A flag indicating the kind of vehicle driven or owned by the offender. The semantics of the values is unknown.
- 'matricola' (string): N/A (Probably refers to the matriculation number of the car.)
- 'notificationType' (string): A flag indicating to whom the fine refers. 'P': car owner, 'C': car driver.
- 'article' (string): The number of the article of the Italian roadtraffic law that is violated by the offender (e.g., article 157 refers to stopping and parking vehicles).
- 'points' (float): Penalty points deducted from the driving license. In Italy, each driver starts with 20 points on their license and may loose points for each offence, based on the gravity.
- 'li

In [203]:
fourth_answer = """To determine how many vehicle classes there are, we need to count the distinct values in the 'vehicleClass' column. This task involves a straightforward query that counts unique entries in a single column. Let's assess this based on the provided heuristics:

1. **Repeated Complex Calculations**: The task involves a simple count of distinct values, which does not require repeated complex calculations or aggregations.

2. **Data Completeness**: The query will account for all cases since it simply counts distinct entries in the 'vehicleClass' column, ensuring no cases are missed.

3. **Simplify Aggregations**: There are no complex aggregations involved, just a distinct count, which is straightforward.

4. **Minimize Logical Steps**: The logic required is minimal, as it involves a single SQL function to count distinct values.

5. **Readability and Maintainability**: The query is inherently simple and readable, involving a single column and a straightforward operation.

6. **Missing Columns**: The 'vehicleClass' column is present, so there is no need to generate additional columns for missing data.

7. **Risk of Direct Queries**: Given the simplicity and direct nature of the task, the risk of error is minimal. The operation is safe to perform directly with an SQL query.

Considering these points, the task is simple, involves minimal logic, and does not require additional columns for clarity or completeness. Therefore, it is safe and efficient to answer this with a direct SQLite query.

**Decision**: Answer with SQLite Query"""

In [204]:
fourth_answer

"To determine how many vehicle classes there are, we need to count the distinct values in the 'vehicleClass' column. This task involves a straightforward query that counts unique entries in a single column. Let's assess this based on the provided heuristics:\n\n1. **Repeated Complex Calculations**: The task involves a simple count of distinct values, which does not require repeated complex calculations or aggregations.\n\n2. **Data Completeness**: The query will account for all cases since it simply counts distinct entries in the 'vehicleClass' column, ensuring no cases are missed.\n\n3. **Simplify Aggregations**: There are no complex aggregations involved, just a distinct count, which is straightforward.\n\n4. **Minimize Logical Steps**: The logic required is minimal, as it involves a single SQL function to count distinct values.\n\n5. **Readability and Maintainability**: The query is inherently simple and readable, involving a single column and a straightforward operation.\n\n6. **Mi

In [205]:
system_prompt= """Determine whether an additional column should be generated or if a question should be directly answered using a SQLite query, with a cautious approach as per the following heuristics:

- **Repeated Complex Calculations**: Generate additional columns if potential queries involve repeated complex calculations or aggregations to store pre-computed values. This reduces complex logic in the query.

- **Data Completeness**: Ensure queries account for all relevant cases, including where certain events do not occur. Pre-compute and store necessary information in a new column if a query might miss cases or events due to filtering.

- **Simplify Aggregations**: Generate columns to store intermediate results (like boolean columns or event counts) if a query requires aggregation, simplifying the final query.

- **Minimize Logical Steps**: Pre-compute values requiring logic to reduce logical steps in queries, making them simpler and less error-prone.

- **Readability and Maintainability**: Generate additional columns if a query becomes difficult to read and maintain due to their length. Simpler queries are easier to debug and less likely to contain logical errors.

- **Risk of Direct Queries**: Only opt for a SQL query directly if it is safe and extremely unlikely to cause an error or logical mistake. Generating additional columns is always a safe bet with no negative consequences. We want to avoid failure as much as possible. Even if we have all the columns required to write a SQLite query, it could be safer to outsource any additional operations to an expert column generator.

- **Missing Columns**: Generate additional columns if questions refer to columns not present in the database, even with close matches (e.g., "amount_min" or "amount_last" if only "amount" is present).

# Steps

1. Analyze the complexity of the potential query and identify any repeated calculations or intricate logic.
2. Check if the potential query covers all relevant data cases, including unoccurring events.
3. Consider the aggregation levels involved and simplify by pre-computing when necessary.
4. Evaluate the logical steps required in the query and minimize them by generating additional columns.
5. Assess the query for readability and maintenance concerns, opting for column generation if it enhances clarity.
6. Verify the presence of necessary columns in the database; generate columns for missing ones referenced in the question.
7. Critically evaluate the risk of directly answering with a SQL query and ensure it is only chosen when truly safe.

# Output Format

Provide a detailed explanation of your assessment followed by your decision: "Generate Additional Columns" or "Answer with SQLite Query".

# Notes

- If the required column is not present in the database, always opt for generating additional columns.
- Consider the maintainability and readability of queries as a factor for decision-making.
- Always provide a clear reasoning process before concluding to justify the decision effectively.
- Be particularly cautious when deciding to opt for a direct SQL query and ensure all safety measures are thoroughly considered."""

In [206]:
system_prompt

'Determine whether an additional column should be generated or if a question should be directly answered using a SQLite query, with a cautious approach as per the following heuristics:\n\n- **Repeated Complex Calculations**: Generate additional columns if potential queries involve repeated complex calculations or aggregations to store pre-computed values. This reduces complex logic in the query.\n\n- **Data Completeness**: Ensure queries account for all relevant cases, including where certain events do not occur. Pre-compute and store necessary information in a new column if a query might miss cases or events due to filtering.\n\n- **Simplify Aggregations**: Generate columns to store intermediate results (like boolean columns or event counts) if a query requires aggregation, simplifying the final query.\n\n- **Minimize Logical Steps**: Pre-compute values requiring logic to reduce logical steps in queries, making them simpler and less error-prone.\n\n- **Readability and Maintainability*